In [ ]:
# Cython, optimización del código

Cómo hacer código Cython [link](http://docs.cython.org/src/quickstart/build.html)

Compilación de código cython [link](http://docs.cython.org/src/reference/compilation.html)

Vemos ahora como se hace en el ipython-notebook

In [1]:
%load_ext  Cython

In [5]:
%%cython_pyximport foo
def f(x):
    return 4.0*x

In [6]:
f(11)

44.0

In [4]:
%%cython
cimport cython
from libc.math cimport exp, sqrt, pow, log, erf

@cython.cdivision(True)
cdef double std_norm_cdf(double x) nogil:
    return 0.5*(1+erf(x/sqrt(2.0)))

@cython.cdivision(True)
def black_scholes(double s, double k, double t, double v,
                 double rf, double div, double cp):
    """Price an option using the Black-Scholes model.
    
    s : initial stock price
    k : strike price
    t : expiration time
    v : volatility
    rf : risk-free rate
    div : dividend
    cp : +1/-1 for call/put
    """
    cdef double d1, d2, optprice
    with nogil:
        d1 = (log(s/k)+(rf-div+0.5*pow(v,2))*t)/(v*sqrt(t))
        d2 = d1 - v*sqrt(t)
        optprice = cp*s*exp(-div*t)*std_norm_cdf(cp*d1) - \
            cp*k*exp(-rf*t)*std_norm_cdf(cp*d2)
    return optprice

In [5]:
black_scholes(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, -1)

10.327861752731728

In [6]:
%timeit black_scholes(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, -1)

1000000 loops, best of 3: 510 ns per loop


In [7]:
%%cython -lm
from libc.math cimport sin
print 'sin(1)=', sin(1)

sin(1)= 0.841470984808


In [44]:
%%cython -lm

cimport cython
from libc.math cimport sin

@cython.cdivision(True)
cdef double f1(double x):
    return sin(x*x)

@cython.cdivision(True)
def f2 (x):
    return f1(x)
  

In [43]:
f2(1.9)


-0.45146575216142315

http://docs.cython.org/src/reference/language_basics.html

search: cython ipython notebook

http://nbviewer.ipython.org/github/ipython/ipython/blob/3607712653c66d63e0d7f13f073bde8c0f209ba8/docs/examples/notebooks/cython_extension.ipynb

http://nbviewer.ipython.org/github/iminuit/iminuit/blob/master/tutorial/hard-core-tutorial.ipynb

http://docs.cython.org/src/userguide/numpy_tutorial.html


In [4]:
%load_ext Cython

In [6]:


%%cython

cimport cython

from libc.stdlib cimport rand, srand
from libc.time cimport time, time_t

cdef extern from "stdlib.h":
    cdef int RAND_MAX

@cython.cdivision(True)
def init_rand ():    
   cdef time_t seconds
   time(&seconds)
   srand(<unsigned int>seconds)
    
@cython.cdivision(True)
def myRand ():
   #return rand()%100 + x
   return <float>rand()/<float>RAND_MAX

In [8]:
init_rand()
for i in range (0,10):
   print(myRand())

0.841356533113867
0.4018294960260391
0.19552548928186297
0.764734631869942
0.36765524512156844
0.7281380812637508
0.5347760589793324
0.6271710558794439
0.8148822844959795
0.05033473018556833


In [34]:
%%cython

cimport cython

import numpy as np
cimport numpy as np


@cython.cdivision(True)
def myRand_np ():
   #return rand()%100 + x
   return np.random.random()

for i in range (0,10):
   print myRand_np()
    


0.881186730034
0.744955958256
0.511022138082
0.901163898904
0.662339827497
0.382745217753
0.889536168513
0.41668731265
0.452326813289
0.536537237168


In [28]:
%timeit myRand_np()

1000000 loops, best of 3: 243 ns per loop


In [33]:
%timeit myRand()

10000000 loops, best of 3: 90.9 ns per loop


In [36]:
%timeit np.random.rand()

1000000 loops, best of 3: 322 ns per loop


In [37]:
import random as rnd

In [38]:
%timeit rnd.random()

10000000 loops, best of 3: 143 ns per loop


In [2]:
%%cython
cimport cython

import numpy as np
cimport numpy as np

import sys

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t


# directivas al compilador
# para mejorar la performance

# Cython asume que no se generarán errores tipo IndexErrors
@cython.boundscheck(False)
# Cython no controla los índices negativos como en python
@cython.wraparound(False)
# Cython asume que si la variable es None, no será accedida
@cython.nonecheck(False)

cdef sum_opt (np.ndarray[DTYPE_t, ndim=2] im1, np.ndarray[DTYPE_t, ndim=2] im2):
    cdef unsigned int rows1 = im1.shape[0]
    cdef unsigned int cols1 = im1.shape[1]
    cdef unsigned int rows2 = im2.shape[0]
    cdef unsigned int cols2 = im2.shape[1]
    
    if rows1 != rows2:
        sys.exit(0)
    if cols1 != cols2:
        sys.exit(0)
    
    for i in range(rows1):
        for j in range(cols1):
            im1[i,j] = im1[i,j] + im2[i,j]
   
 
def test_sum():
    cdef np.ndarray[DTYPE_t, ndim=2] im1
    cdef np.ndarray[DTYPE_t, ndim=2] im2
    
    im1 = np.ones(5*5).reshape(5,5)
    im2 = np.ones(5*5).reshape(5,5)
    sum_opt(im1, im2)
    print(im1)
    
def main():    
    test_sum()
main()

[[ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.]]


# Optimizar el código de una clase con Cython

In [3]:
%%cython
cimport cython

import numpy as np
cimport numpy as np

import sys

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# directivas al compilador
# para mejorar la performance

# Cython asume que no se generarán errores tipo IndexErrors
@cython.boundscheck(False)
# Cython no controla los índices negativos como en python
@cython.wraparound(False)
# Cython asume que si la variable es None, no será accedida
@cython.nonecheck(False)

cdef sum_opt (np.ndarray[DTYPE_t, ndim=2] im1, np.ndarray[DTYPE_t, ndim=2] im2, DTYPE_t p):
    cdef unsigned int rows1 = im1.shape[0]
    cdef unsigned int cols1 = im1.shape[1]
    cdef unsigned int rows2 = im2.shape[0]
    cdef unsigned int cols2 = im2.shape[1]
    
    if rows1 != rows2:
        print("error dimensiones filas")
        sys.exit(0)
    if cols1 != cols2:
        print("error dimensiones columnas")
        sys.exit(0)
    
    for i in range(rows1):
        for j in range(cols1):
            im1[i,j] = im1[i, j] * (1-p) + p*im2[i, j]
            
            
class damero_opt():  
    def __init__(self, w=100, h=100, i=255):
        self.h = h
        self.w = w 
        self.img = i * np.ones(h*w).reshape(h,w)
        
    def add_level(self, l=0):
        self.img = self.img + l
        self.img[self.img > 255] = 255
        self.img[self.img < 0] = 0
    
    def sum(self, damero, p=0.5):
        sum_opt(self.img, damero.img, p)
    
d1 = damero_opt(w=64, h = 36, i=1)
d2 = damero_opt(w=64, h = 36, i=30)
d1.sum(d2)